In [2]:
import numpy as np

In [56]:
x = np.array([0, 0, -0.068, -0.137, -0.137, -0.137, -0.162])
y = np.array([0, 0, 0, 0, 0.2, 0.39, 0.463])
z = np.array([0.12, 0.32, 0.509, 0.696, 0.7, 0.7, 0.687])
roll = np.array([0, 0, 0.349, 1.569, -1.57, -1.694, -1.694])
pitch = np.array([0, -0.349, 0, -1.222, -0.348, -0.362, -0.326])
yaw = np.array([0, 0, -1.571, -3.14, 0, 0.372, 0.372])

axes = np.array([[0, 0, 1], [0, -1, 0], [0, 0, 1], [0, 1, 0], [0, 0, 1], [0, -1, 0], [0, 0, 1]])
origins = np.array([x, y, z]).T
angles_rpy = np.array([roll, pitch, yaw]).T

In [57]:
def rotate(angles_rpy, vector):
    gamma = angles_rpy[0]
    beta = angles_rpy[1]
    alpha = angles_rpy[2]
    b = vector / np.linalg.norm(vector)
    
    sin = np.sin
    cos = np.cos
    
    a = np.array([[cos(alpha)*cos(beta), cos(alpha)*sin(beta)*sin(gamma) - sin(alpha)*cos(gamma), cos(alpha)*sin(beta)*cos(gamma) + sin(alpha)*sin(gamma)],
                 [sin(alpha)*cos(beta), sin(alpha)*sin(beta)*sin(gamma) + cos(alpha)*cos(gamma), sin(alpha)*sin(beta)*cos(gamma) - cos(alpha)*sin(gamma)],
                 [-sin(beta), cos(beta)*sin(gamma), cos(beta)*cos(gamma)]])
    
    c = a.dot(b)
    
    return c / np.linalg.norm(c)


def build_jacobian(origins, angles_rpy, axes, endeffector=origins[-1], offset=0):
    j = np.array([np.concatenate([np.cross(rotate(angle, axis), endeffector - origin), rotate(angle, axis)])
                  for origin, angle, axis in zip(origins[offset:], angles_rpy[offset:], axes[offset:])])
    
    return j.T

In [58]:
jacobian = build_jacobian(origins, angles_rpy, axes)

np.set_printoptions(precision=3, linewidth=120)
print(jacobian)

[[-4.630e-01 -3.670e-01 -4.351e-01 -1.582e-01 -1.320e-02 -6.758e-02  0.000e+00]
 [-1.620e-01 -0.000e+00 -2.746e-02 -8.609e-05 -2.225e-05 -2.804e-02  0.000e+00]
 [ 0.000e+00 -1.620e-01 -1.583e-01  4.351e-01  2.493e-02 -2.749e-02 -0.000e+00]
 [ 0.000e+00  0.000e+00 -3.420e-01  9.398e-01 -2.716e-04 -3.721e-01 -3.241e-01]
 [ 0.000e+00 -1.000e+00  6.965e-05 -2.996e-04  1.000e+00 -1.326e-02  9.388e-01]
 [ 1.000e+00  0.000e+00  9.397e-01  3.418e-01  7.486e-04  9.281e-01 -1.164e-01]]


In [60]:
force = np.concatenate([rotate(angles_rpy[-1], [0, 0, 1]), [0, 0, 0]])
torque = jacobian.T.dot(force)

print('Force vector: {}'.format(force))
print('Joint torques: {}'.format(torque))

Force vector: [-0.324  0.939 -0.116  0.     0.     0.   ]
Joint torques: [-0.002  0.138  0.134  0.001  0.001 -0.001  0.   ]


In [62]:
# Hand to base transformation
ee = np.array([rotate(angles_rpy[-1], axis) for axis in ([1, 0, 0], [0, 1, 0], [0, 0, 1])])
htb = np.zeros((6, 6))
htb[0:3,0:3] = htb[3:,3:] = np.linalg.inv(ee)
jacobian_htb = htb.dot(jacobian)
torque = jacobian.T.dot(np.array([0, 0, 1, 0, 0, 0]))

torque

array([ 0.   , -0.162, -0.158,  0.435,  0.025, -0.027,  0.   ])